In [ ]:
import os
import pandas as pd

def split_textgrid_intervals(input_file, output_s, output_t, excel_s, excel_t):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    intervals_s = []
    intervals_t = []

    excel_data_s = []
    excel_data_t = []

    current_interval = {}
    sequence_s = 1
    sequence_t = 1
    file_base = os.path.splitext(os.path.basename(input_file))[0]  # Extract file name without extension

    for line in lines:
        line = line.strip()
        if line.startswith("xmin ="):
            current_interval["xmin"] = float(line.split("=")[1].strip())
        elif line.startswith("xmax ="):
            current_interval["xmax"] = float(line.split("=")[1].strip())
        elif line.startswith("text ="):
            text = line.split("=")[1].strip().strip('"')
            current_interval["text"] = text

            if text == "Student":
                intervals_s.append(current_interval.copy())
                excel_data_s.append([sequence_s, current_interval["xmin"], current_interval["xmax"], "S", f"{file_base}s{sequence_s}"])
                sequence_s += 1
            elif text == "Interviewer":
                intervals_t.append(current_interval.copy())
                excel_data_t.append([sequence_t, current_interval["xmin"], current_interval["xmax"], "T", f"{file_base}t{sequence_t}"])
                sequence_t += 1

            current_interval = {}

    def write_intervals(filename, intervals):
        if not intervals:
            return  # Skip writing if no intervals exist
        with open(filename, 'w') as file:
            file.write("File type = \"ooTextFile\"\n")
            file.write("Object class = \"TextGrid\"\n\n")
            file.write("xmin = 0\n")
            file.write("xmax = {}\n".format(intervals[-1]["xmax"]))
            file.write("tiers? <exists>\n")
            file.write("size = 1\n")
            file.write("item []:\n")
            file.write("    item [1]:\n")
            file.write("        class = \"IntervalTier\"\n")
            file.write("        name = \"extracted_intervals\"\n")
            file.write("        xmin = 0\n")
            file.write("        xmax = {}\n".format(intervals[-1]["xmax"]))
            file.write("        intervals: size = {}\n".format(len(intervals)))

            for i, interval in enumerate(intervals, start=1):
                file.write("        intervals [{}]:\n".format(i))
                file.write("            xmin = {}\n".format(interval["xmin"]))
                file.write("            xmax = {}\n".format(interval["xmax"]))
                file.write("            text = \"{}\"\n".format(interval["text"]))

    write_intervals(output_s, intervals_s)
    write_intervals(output_t, intervals_t)

    def write_excel(filename, data):
        if data:
            df = pd.DataFrame(data, columns=["Number", "Start time", "End time", "Member", "File Name"])
            df.to_excel(filename, index=False)

    write_excel(excel_s, excel_data_s)
    write_excel(excel_t, excel_data_t)

# Usage example:
split_textgrid_intervals("230.TextGrid", "output_s.TextGrid", "output_t.TextGrid", "output_s.xlsx", "output_t.xlsx")

In [ ]:
!pip install pydub
!apt-get install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [ ]:
import os
from pydub import AudioSegment

def split_textgrid_audio(textgrid_file, audio_file):
    # Read the TextGrid file
    with open(textgrid_file, 'r') as file:
        lines = file.readlines()

    intervals_s = []
    intervals_t = []
    current_interval = {}

    for line in lines:
        line = line.strip()
        if line.startswith("xmin ="):
            current_interval["xmin"] = float(line.split("=")[1].strip())
        elif line.startswith("xmax ="):
            current_interval["xmax"] = float(line.split("=")[1].strip())
        elif line.startswith("text ="):
            text = line.split("=")[1].strip().strip('"')
            current_interval["text"] = text
            if text == "Student":
                intervals_s.append(current_interval.copy())
            elif text == "Teacher":
                intervals_t.append(current_interval.copy())
            current_interval = {}

    # Load the audio file
    audio = AudioSegment.from_file(audio_file, format="mp3")
    base_name = os.path.splitext(os.path.basename(audio_file))[0]

    # Function to save split audio
    def save_intervals(intervals, label):
        output_dir = f"{base_name}_{label}"
        os.makedirs(output_dir, exist_ok=True)

        for i, interval in enumerate(intervals, start=1):
            start_ms = int(interval["xmin"] * 1000)
            end_ms = int(interval["xmax"] * 1000)
            segment = audio[start_ms:end_ms]
            filename = f"{base_name}{label}{i}.mp3"
            file_path = os.path.join(output_dir, filename)
            segment.export(file_path, format="mp3")

    save_intervals(intervals_s, "s")
    save_intervals(intervals_t, "t")

# Usage Example
# this will now work since we have installed pydub and ffmpeg
split_textgrid_audio("/content/230.TextGrid", "/content/230.mp3")

In [ ]:
import librosa

for i in range(1, 11):  # Loop from 1 to 10 (assuming you have 230s1.mp3 to 230s10.mp3)
    file_name = f'/content/230s{i}.mp3'  # Adjusted filename pattern
    audio_ts_var = f'audio_ts{i}'  # Construct audio_ts variable name
    sampling_rate_var = f'sampling_rate{i}'  # Construct sampling_rate variable name

    try:
        # Load audio file
        audio_data, sampling_rate = librosa.load(file_name)

        # Assign loaded data and sampling rate
        exec(f'{audio_ts_var} = audio_data')
        exec(f'{sampling_rate_var} = sampling_rate')

        # Extract MFCC features
        mfcc_feats = librosa.feature.mfcc(y=audio_data, sr=sampling_rate, n_mfcc=13)

        # Print features
        print(f'MFCC features for {file_name}:')
        print(mfcc_feats)

    except Exception as e:
        print(f"Error loading {file_name}: {e}")


MFCC features for /content/230s1.mp3:
[[-5.8009607e+02 -5.5846173e+02 -5.5658527e+02 ... -4.9592535e+02
  -5.0780856e+02 -5.2507349e+02]
 [ 1.5250571e+02  1.6536327e+02  1.6671268e+02 ...  1.6730110e+02
   1.6578499e+02  1.5721524e+02]
 [ 6.9559326e+01  6.0363350e+01  5.8614540e+01 ...  1.0063000e+01
   1.9942717e+01  3.0112705e+01]
 ...
 [ 6.7735882e+00  4.0837088e+00 -1.9130341e+00 ...  1.0842068e+01
   5.1047931e+00  2.1223323e+00]
 [ 2.7901773e+00  3.3591378e-01 -2.7697392e+00 ... -1.4522812e+01
  -1.2852325e+01 -4.9036846e+00]
 [ 1.9808655e+00  8.5196006e-01  5.9174550e-01 ... -8.2380028e+00
  -5.6696906e+00 -7.9929199e+00]]
MFCC features for /content/230s2.mp3:
[[-5.4266992e+02 -5.4494965e+02 -5.6068347e+02 ... -5.4070886e+02
  -5.2866559e+02 -5.0754782e+02]
 [ 1.4529192e+02  1.5831679e+02  1.5592007e+02 ...  1.7244135e+02
   1.6865057e+02  1.4772643e+02]
 [ 4.1930752e+01  5.5727497e+01  6.2074486e+01 ...  6.2516579e+01
   5.2078503e+01  3.5310860e+01]
 ...
 [-2.0092454e+00  3.53

In [ ]:
import librosa
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import IPython.display as ipd

# Step 1: Load and process multiple audio files
mfcc_features_list = []
processed_files = []

for i in range(1, 11):  # Loop from 1 to 10
    file_name = f'/content/230s{i}.mp3'

    try:
        # Load audio file
        audio_data, sampling_rate = librosa.load(file_name, sr=16000)

        # Check if the file is silent
        if np.all(audio_data == 0):
            print(f"Warning: {file_name} is silent.")
            continue  # Skip this file

        # Extract MFCC features
        mfcc_feats = librosa.feature.mfcc(y=audio_data, sr=sampling_rate, n_mfcc=13)
        mfcc_feats = np.mean(mfcc_feats, axis=1)  # Take mean over time

        # Store valid features
        mfcc_features_list.append(mfcc_feats)
        processed_files.append(file_name)

    except Exception as e:
        print(f"Error loading {file_name}: {e}")

# Convert MFCC features to DataFrame
mfcc_features_df = pd.DataFrame(mfcc_features_list)

# Step 2: Load labels from the Excel file
file_path = "/content/output_s.xlsx"
df = pd.read_excel(file_path)

# Extract only the first 10 rows of the "Fluency" column
if "Fluency" not in df.columns:
    raise ValueError("Error: 'Fluency' column not found in the Excel file.")

y = df["Fluency"].iloc[:10].values  # Select only the first 10 values

# Debugging Output: Check sizes
print(f"Processed audio files: {len(mfcc_features_list)}")
print(f"Fluency labels used: {y}")

# Ensure labels match the number of audio files
if len(y) != len(mfcc_features_list):
    raise ValueError(f"Mismatch: {len(mfcc_features_list)} audio samples but {len(y)} Fluency labels.")

# Step 3: Prepare features and labels
X = mfcc_features_df.values  # Features

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 4: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 5: Train a Classification Model (Random Forest)
clf = RandomForestClassifier(n_estimators=500, random_state=42)
clf.fit(X_train, y_train)

# Step 6: Make Predictions
y_pred = clf.predict(X_test)

# Step 7: Evaluate the Model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred))




Processed audio files: 10
Fluency labels used: [1. 1. 0. 0. 0. 1. 0. 1. 1. 0.]
Accuracy: 0.00
Classification Report:
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       0.0
         1.0       0.00      0.00      0.00       2.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_